Create a simple chatbot that answers questions based on your documents using Azure OpenAI and Azure Cognitive Search.

In [ ]:
!pip install openai azure-search-documents python-dotenv


In [ ]:
import os

# Azure OpenAI
os.environ["AZURE_OPENAI_KEY"] = "abc123fakeopenaikey987xyz"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://my-openai-resource.openai.azure.com/"
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "gpt-35-turbo"

# Azure Cognitive Search
os.environ["AZURE_SEARCH_KEY"] = "xyz456fakesearchkey123abc"
os.environ["AZURE_SEARCH_ENDPOINT"] = "https://my-search-resource.search.windows.net"
os.environ["AZURE_SEARCH_INDEX"] = "my-index"

In [ ]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
import openai

In [ ]:
openai.api_key = os.environ["AZURE_OPENAI_KEY"]
openai.api_base = os.environ["AZURE_OPENAI_ENDPOINT"]
openai.api_version = "2023-05-15"
deployment_name = os.environ["AZURE_OPENAI_DEPLOYMENT"]


In [ ]:
search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

In [ ]:
def ask_rag_question(question):
    search_results = search_client.search(question)
    docs = [doc['content'] for doc in search_results]
    context = "\n".join(docs[:3])


    prompt = f"Use the context below to answer the question:\n\n{context}\n\nQuestion: {question}"

    response = openai.ChatCompletion.create(
        engine=deployment_name,
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

In [ ]:
question = "What are the company policies on remote work?"
answer = ask_rag_question(question)

print("📨 Question:", question)
print("🤖 Answer:", answer)


In [ ]:
extended_path = Path("/mnt/data/azure_rag_colab_extended.py")
extended_path.write_text(extended_code)

# Add to ZIP
with zipfile.ZipFile("/mnt/data/azure_rag_chatbot_colab.zip", 'a') as zipf:
    zipf.write(extended_path, arcname="azure_rag_colab_extended.py")

"/mnt/data/azure_rag_chatbot_colab.zip"